#### Year 2 Computational Challenge - Machine Learning
    Douglas Penning, Roman Ogorodnov, Manish Saharan, Ryan Hutchings
    
Overview:

We chose to break the programme into key sections, detailed by the headings shown throught:
- Import & Preprocessing

    - Data is imported from the .csv file, with the number of duplicate, and null entries counted. Removing the **23 duplicate entries** was found to improve the RMSE of the chosen model by **~0.5**. We chose to initially use **simple median imputer** and **standard scaler** to ensure a consistent and computationally simple approach allowing each model to be tested using the same datasets.


    - We chose to use an 80-20 test-train split and created the global variables **gX_train_preprocessed, gy_test etc** which are used throughout

    - **Note:** While as a group we did discuss the posability of removing variables initially, while we accepted there was likley a correlation between many of them eg course & fine aggregate, we felt that it would be a more rigourous approach to iteratively test each dropped combination and then retrain a final model using the new dataset if required

- Model selection

    - We begin by defining each regression function (using a function for each allowed us to quickly individually test & efficiently pass through hyperparameters etc), using the standard parameters in scikit-learn and thus having the default best_params = {}. After each regressor is created the global model name dictionary is defined

    - Models were tested unoptimised to reduce computational expense, we chose to use 3 error types:
        * **Root Mean Square Error** - Giving differences between actual and predicted values and actual, with the ^2 giving greater weighting to larger errors.
        * **R^2** - Measuring correlation between datapoints, used as it is easily visually observed in the plots 
        * **Mean Absolute Average Error** - While similar to RMSE, MAE is less sensitive to larger outliers*
    - Models were then ranked on each error type (accounting for the possability for models performing better/worse in specific error types) with **Random Forrest** achieving the best score. **Note:** We have tried to ensure this code is resilient, and as such have used best_model and the model dictionary where possible in case a different dataset is better suited to a different model (though this was not possible in the optimisation section).

- Optimisation
    - We employed randomised serch initially though found that results were highly variable so chose GridSearch. Initial runs were carried out on larger prameter grids, taking many hours to run. The grid used in this code covers the best parameters found.
        
- Dropping variables
    - We chose to drop all possible combinations of variables using an 8 bit binary number as an index, allowing us to analyse any correlation and it's effect on model performance. Depending on model optimisation it was found that removing course & fine aggregates produced improved results.

    - 2 datafiles are produced, showing the binary drop_index and errors, and a second *.xlsx* using the variable names (for single and pairs) allowing easier analysis.

- Graphing & Final data presentation
    - Comparitive graphs are shown for each regression model, allowing quick visual comparison 
    - An interactive line graph, for the chosen model, allows the user to deselect  variables to see the effect on correlation and errors quickly. This uses the gVarErrorDF meaning no extra computation is required
    - An interactive bar chart shows the effect removing individual errors has on each error metric, with a baseline shown in red

### Importing libraries
This section imports all libraries utilised within the code:
 - Misc including pandas & numpy etc.
 - scikit-learn modules
 - Graphing libraries including matplotlib, as well as ipywidgets and seaborn for interactivity/formatting

In [ ]:
import pandas as pd
import numpy as np
from math import sqrt
from scipy.stats import norm

# SKLearn
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import learning_curve


# Graphing
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from ipywidgets import interact, widgets, interactive
from matplotlib.ticker import AutoMinorLocator



# Global Variables
global gMethodDictionary
global gVarErrorDf
gVarErrorDf = pd.DataFrame(columns=[ 'Drop index','Predicted y', 'RMSE', 'R^2', 'MAE', 'RMSE %', 'R^2 %', 'MAE %'])
gVarNames = ['Cement','Blast Furnace Slag','Fly Ash','Water','Superplasticizer','Coarse Aggregate','Fine Aggregate','Age','Concrete compressive strength',' ']
gX_train_preprocessed = pd.DataFrame()
gX_test_preprocessed = pd.DataFrame()
gy_train = []
gy_test = []


### Data import, handling and preprocessing
 - Imports, drops duplicated rows, seperates training/target columns, completes basic preprocessing (see overview)

In [ ]:
def csv_import():
    #import data from the files
    dataset = pd.read_csv('Concrete_Data_Yeh_final.csv')

    #Data Preprocessing
    #format as a dataframe
    dataset = pd.DataFrame(dataset)
    
    print(f'Null values: \n', dataset.isnull().sum()) #check for null values
    print(f'Duplicated values:',dataset.duplicated().sum()) #check for duplicates
    dataset = dataset.drop_duplicates() #drop duplicates
    dataset.dtypes #check for data types
  
    y = dataset["csMPa"]
    X = dataset.drop("csMPa", axis=1)

    return X, y

In [ ]:
def preprocessing(X,y):
    # Splitting the data into training and test sets
    global gX_train_preprocessed, gX_test_preprocessed
    global gy_train, gy_test    

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Creating a preprocessing pipeline that imputes missing values with the mean 
    # and scales features to have zero mean and unit variance.
    preprocessing_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())])

    gX_train_preprocessed = pd.DataFrame(preprocessing_pipeline.fit_transform(X_train))
    gX_test_preprocessed = pd.DataFrame(preprocessing_pipeline.transform(X_test))
    gy_train = y_train
    gy_test = y_test

## Regression functions

This section defines functions for various regression models which are later compared to choose the most effective model.

The regression models included are:

1. **Linear Regression (`linear_regression`)**: While it is stated that data is nonlinear we used it as a baseline.

2. **Decision Tree Regression (`decision_tree_regression`)**

3. **Random Forest Regression (`random_forest_regression`)**: **Chosen model**

4. **Lasso Regression (`lasso_regression`)**

5. **Elastic Net Regression (`elastic_net_regression`)**

6. **Ridge Regression (`ridge_regression`)**

7. **Support Vector Regression (`svr_regression`)**

8. **K-Nearest Neighbors Regression (`knn_regression`)**



In [ ]:
def linear_regression(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a linear regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 
   
    regressor = LinearRegression(**bestFit) # Creating the Linear Regression model
    regressor.fit(xTrainData, yTrainData) # Fitting the data
    y_pred = regressor.predict(yTestData) # Making predictions

    return y_pred

In [ ]:
def decision_tree_regression(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a decision tree regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 

    regressor = DecisionTreeRegressor(**bestFit) # Creating the Decision Tree regressor
    regressor.fit(xTrainData, yTrainData)
    y_pred = regressor.predict(yTestData)
    
    return y_pred

In [ ]:
def random_forest_regression(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a random forest regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 
   
    regressor = RandomForestRegressor(**bestFit) # Creating the Random Forest Regressor
    regressor.fit(xTrainData, yTrainData) 
    y_pred = regressor.predict(yTestData)

    return y_pred

In [ ]:
def perform_lasso(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a Lasso regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 
    
    model = Lasso(**bestFit) # Create and train the model
    model.fit(xTrainData, yTrainData)
    y_pred = model.predict(yTestData)
    
    return y_pred

In [ ]:
def elastic_net_regression(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a elastic net regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 

    regressor = ElasticNet(**bestFit)    # Creating the Elastic Net Regressor
    regressor.fit(xTrainData, yTrainData)
    y_pred = regressor.predict(yTestData)

    return y_pred

In [ ]:
def ridge_regression(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a ridge regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 

    regressor = Ridge(**bestFit) # Creating the Ridge Regressor
    regressor.fit(xTrainData, yTrainData)
    y_pred = regressor.predict(yTestData)

    return y_pred

In [ ]:
def perform_svr(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a support vector regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 
   
    model = SVR(**bestFit)  # Create and train the model
    model.fit(xTrainData, yTrainData)
    y_pred = model.predict(yTestData)

    return y_pred

In [ ]:
def perform_knn(xTrainData: pd.DataFrame, yTrainData: pd.DataFrame, yTestData: pd.Series, bestFit = {}):
    """
    This function creates and fits a KNN regression model

    Parameters:
    xTrainData (numpy.ndarray or pandas.DataFrame): The independent variables for training
    yTrainData (numpy.ndarray or pandas.Series): Target training values (Compressive Strength)
    yTestData (numpy.ndarray or pandas.DataFrame): The independent variables for the test set, used to predict y_pred
    bestFit (dict, optional): The parameters for the Linear Regression model. Defaults to an empty dictionary.

    Returns:
    numpy.ndarray: The predicted values of the dependent variable for the test set.
    """ 

    model = KNeighborsRegressor(**bestFit)    # Create and train the model
    model.fit(xTrainData, yTrainData)
    y_pred = model.predict(yTestData)
    
    return y_pred

In [ ]:
# Defines a dictionary containing Name:Function pairs for each regression method
gMethodDictionary = [('Linear Regression', linear_regression),
                    ('Decision Tree Regression', decision_tree_regression),
                    ('Random Forest Regression', random_forest_regression),
                    ('Ridge', ridge_regression), 
                    ('Lasso', perform_lasso),
                    ('Elastic Net', elastic_net_regression),
                    ('Support Vector Regression', perform_svr),
                    ('K-Nearest Neighbors Regression', perform_knn)]

## Model Evaluation

Analysing the performance of each regression model to determine which technique most accuratley predicts compressive strength

    

In [ ]:
def regression_performance():
    """
    This function evaluates the performance of all regression models defined, iterating through the global `gMethodDictionary`.
    Predicted values are used to calculate the root mean squared error (RMSE), R^2 score, and mean absolute error (MAE).
    The results are stored in a DataFrame.

    Models are ranked based on each performance metric, with the highest R^2 score and the lowest 
    RMSE and MAE being the best. It calculates a rank sum for each model and identifies the model with the lowest 
    rank sum as the best model.

    Parameters:
    None

    Returns:
    best_model (str): The name of the best model.
    df (pd.DataFrame): A DataFrame containing the performance metrics and ranks for each model.
    """
    
    data = []
    for model_name, model in gMethodDictionary: # Loop over each model
        # Calling each regression model, passing in the training and test data (Note: models are unoptimised so bestFit assumes default parameters))      
        y_pred = model(gX_train_preprocessed, gy_train, gX_test_preprocessed)

        # Calculate the performance metrics
        rmse = sqrt(mean_squared_error(gy_test, y_pred))
        r2 = r2_score(gy_test, y_pred)
        mae = mean_absolute_error(gy_test, y_pred)
    
        # Append the data to the list
        data.append([model_name, y_pred, rmse, r2, mae])

    # Create a DataFrame from the data
    df = pd.DataFrame(data, columns=['Model', 'y_pred', 'RMSE', 'R^2','MAE'])

    # Set the model names as the index
    df.set_index('Model', inplace=True)

    # Rank each metric, with the highest being the best for R^2 and the lowest being the best for RMSE and MAE
    df['R^2_rank'] = df['R^2'].rank(ascending=False)
    df['RMSE_rank'] = df['RMSE'].rank()
    df['MAE_rank'] = df['MAE'].rank()

    # Calculate the sum of the ranks
    df['rank_sum'] = df['R^2_rank'] + df['RMSE_rank'] + df['MAE_rank'] 

    # Find the model with the lowest rank sum
    best_model = df['rank_sum'].idxmin()

    return best_model, df

## Hyperperameters

As shown Random Forrest Regression has the best untuned performance on our dataset and as such we are only analysing hyperperameters for this model:
The most important hyperparameters to tune in a Random Forest model to improve its performance are:

1. `n_estimators`: This is the number of trees you want to build before taking the maximum voting or averages of predictions. Higher number of trees give you better performance but makes your code slower.

2. `max_depth`: The maximum depth of the tree. This parameter can help to prevent overfitting. If the max depth is too high, the model may learn too much from the training data and perform poorly on unseen data.

3. `min_samples_split`: The minimum number of samples required to split an internal node. If you increase this parameter, each tree in the forest becomes more constrained as it has to consider more samples at each node.

4. `min_samples_leaf`: The minimum number of samples required to be at a leaf node. This parameter is similar to min_samples_splits, however, this describe the minimum number of samples of samples at the leafs, the base of the tree.

5. `max_features`: The number of features to consider when looking for the best split. If set to "auto", then `max_features=sqrt(n_features)`.

6. `bootstrap`: Whether bootstrap samples are used when building trees. If False, the whole dataset is used to build each tree.

Both Random Search and Grid Search are hyperparameter tuning techniques, and each has its own advantages and disadvantages.

**Grid Search** systematically works through multiple combinations of parameter tunes, cross-validating as it goes to determine which tune gives the best performance. The benefit is that it's guaranteed to find the best combination of parameters supplied. However, it can be computationally expensive, especially if the number of parameters or their possible values are large.

**Random Search** sets up a grid of hyperparameter values and selects random combinations to train the model and score. The benefit is that it's not as computationally expensive as Grid Search, and you have more control over how long you want it to run for, as you can set the number of iterations. However, it's not guaranteed to find the best parameters.

In [ ]:
def grid_search():
    """
    This function performs a grid search to identify the best hyperparameter combination for the Random Forest Regressor model.
    The function first defines a hyperparameter grid, then performs a search to identify the best hyperparameter combination. 

    Parameters:
    None

    Returns:
    best_grid_params (dict): The best hyperparameter combination.
    """

    #Parameters used for initial long run shown below, initial run took over 12 hours to complete so parameters were narrowed to a range covering the 
    #best parameters and shown to demonstrate the process. This was tested using randomised search as the full grid search would compute 1,135,104 combinations taking over
    #15 days to complete.
    """ 
    param_grid = {
        'n_estimators': [50, 100, 125, 135, 135, 155, 165, 200, 300],
        'max_depth': [None,5, 20, 30, 40, 50],
        'min_samples_split': [ 1, 2, 3, 4, 5, 10],
        'min_samples_leaf': [ 1, 2, 5, 10],
        'max_features': ['auto', 'sqrt'],
        'bootstrap': [True, False],
        'max_leaf_nodes': [None, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90],
        'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
    }"""
    # With this parameter grid, optimised values have been found to be:
    # Optimized RMSE: 4.081712201661637
    # Optimized R^2: 0.9273305050035258
    # Optimized MAE: 2.9955957095709524

    param_grid = {
        'n_estimators': [150, 155, 160],
        'max_depth': [25, 30, 35],
        'min_samples_split': [2, 3, 4],
        'max_features': ['auto', 'sqrt'],
        'bootstrap': [False]}

    # Step 5: Perform grid search
    rf_grid = GridSearchCV(estimator=RandomForestRegressor(), param_grid=param_grid, cv=5, verbose = 1)
    rf_grid.fit(gX_train_preprocessed, gy_train)

    # Step 6: Identify the best hyperparameter combination from grid search
    best_grid_params = rf_grid.best_params_

    return best_grid_params


In [ ]:
def opt_unopt_random_forest(best_grid_params: dict):
    """
    This function compares the performance of the original unoptimised Random Forest Regressor model with the optimised Random Forest Regressor model.
    The model trained, using the hyperparameter combination identified by the grid search, performance metrics are compared to the original model. If the
    optimised model performs worse than the original model, the grid search is deemed unsuccessful and an empty dictionary is returned.

    Parameters:
    None

    Returns:
    best_grid_params (dict): The best hyperparameter combination.
    """ 
    # Original unoptimized Random Forest Regression
    y_pred_unoptimized = random_forest_regression(gX_train_preprocessed, gy_train, gX_test_preprocessed)

    # Optimized Random Forest Regression  
    y_pred_optimized = random_forest_regression(gX_train_preprocessed, gy_train, gX_test_preprocessed,best_grid_params)

    # Calculate errors
    rmse_unoptimised = sqrt(mean_squared_error(gy_test, y_pred_unoptimized))
    rmse_optimised = sqrt(mean_squared_error(gy_test, y_pred_optimized))
    r2_unoptimised = r2_score(gy_test, y_pred_unoptimized)
    r2_optimised = r2_score(gy_test, y_pred_optimized)
    mae_unoptimised = mean_absolute_error(gy_test, y_pred_unoptimized)
    mae_optimised = mean_absolute_error(gy_test, y_pred_optimized)

    # Print errors
    print("Unoptimized RMSE:", rmse_unoptimised)
    print("Optimized RMSE:", rmse_optimised)
    print("Unoptimized R^2:", r2_unoptimised)
    print("Optimized R^2:", r2_optimised)
    print("Unoptimized MAE:", mae_unoptimised)
    print("Optimized MAE:", mae_optimised)

    if rmse_optimised > rmse_unoptimised:
        print("Optimisation unsuccessful")
        best_grid_params = {}
    return best_grid_params

## Dropping variables

    var_drop():
Used to create a new data set, dropping all combinations of variables using a binary 'drop index'. This allows us to determine the effect on regression performance when dropping single, pairs, triplets or any other combination of variables. This was chosen to allow us to clearly see and account for variable correlation, and efficiently plot the interactive graphs as shown.

    var_performance():
Taking the dataset from var_drop(): to measure the performance effect of dropping variables

    var_impact_pct():
Calculating the difference in original vs retrained performance for various combinations

In [ ]:
def var_drop(dropVar: int):
    """
    Drops columns from the training and testing datasets based on a binary number.

    Each digit in the binary number corresponds to a column in the datasets. If the digit is 1, the corresponding column is dropped. If the digit is 0, the corresponding column is kept.

    Parameters:
    dropVar (int): An intiger, converted to binary with each digit representing a column in the datasets.

    Returns:
    datasetTrain_drop (pd.DataFrame): The training dataset with the columns dropped.
    datasetTest_drop (pd.DataFrame): The testing dataset with the columns dropped.
    """

    # Convert the binary number to a list of booleans
    bool_list = [bool(int(x)) for x in format(dropVar, '0{}b'.format(len(gX_train_preprocessed.columns)))]

    # Create a list of column names to drop based on the boolean list
    columns_to_drop_train = gX_train_preprocessed.columns[bool_list]
    columns_to_drop_test = gX_test_preprocessed.columns[bool_list]

    # Drop the columns
    datasetTrain_drop = gX_train_preprocessed.drop(columns_to_drop_train, axis=1)
    datasetTest_drop = gX_test_preprocessed.drop(columns_to_drop_test, axis=1)

    return datasetTrain_drop, datasetTest_drop

In [ ]:
def var_performance(best_grid_params: dict):
    """
    This function evaluates the performance of the best regression model for every combination of dropped variables.

    The function first identifies the best regression model by calling the `regression_performance` function. 
    It then loops over all variables in the preprocessed training and testing data, dropping one variable at a time 
    and evaluating the performance of the model without that variable. 

    The function also calculates the variation of each performance metric compared to the base case (i.e., when no 
    variables are dropped). The results are stored in the global DataFrame `gVarErrorDf`.

    Parameters:    None
    Returns:    None
    """
    x_train_drop = gX_train_preprocessed
    x_test_drop = gX_test_preprocessed
    best_model, df = regression_performance() # Replace with a pass in parameter

    for key, model_function in gMethodDictionary:
        if key == best_model:
            break

    for i in range(255):
        x_train_drop, x_test_drop = var_drop(i)
       
        y_pred = model_function(x_train_drop, gy_train, x_test_drop, best_grid_params)
        rms_error = sqrt(mean_squared_error(gy_test, y_pred))
        r2_error = r2_score(gy_test, y_pred)
        ma_error = mean_absolute_error(gy_test, y_pred)
        if i == 0:
            rms_var = 1
            r2_var = 1
            ma_var = 1
            rms_base = rms_error
            r2_base = r2_error
            ma_base = ma_error

        else:
            rms_var = rms_error / rms_base
            r2_var = r2_error / r2_base
            ma_var = ma_error / ma_base


        gVarErrorDf.loc[i] = [bin(i), y_pred, rms_error, r2_error, ma_error, rms_var, r2_var, ma_var]

In [ ]:
def var_impact_pct():
    """
    This function calculates the impact of each variable on the model's performance metrics (RMSE, R^2, and MAE) 
    when the variable is dropped. It creates a DataFrame that contains the performance metrics for each variable 
    when it's dropped, as well as when no variables are dropped. It also calculates the impact of dropping two 
    variables at a time. 

    The function then ranks each model (i.e., each set of dropped variables) based on the performance metrics, 
    calculates a rank sum for each model, and sorts the DataFrame by the rank sum. The model with the lowest rank 
    sum is considered the best model.

    The DataFrame is printed to the console and also exported to an Excel file named 'impact_df.xlsx'.

    Parameters:    None

    Returns:    None
    """

    impact_df = pd.DataFrame(columns=['Dropped variable: 1','Dropped variable : 2', 'RMSE', 'R^2', 'MAE','RMSE %', 'R^2 %', 'MAE %', ])

    #Single variable impact
    for i in range(8):
        index = 2**i
        impact_df.loc[i] = [gVarNames[(7-i)],gVarNames[9], gVarErrorDf.loc[index]['RMSE'], gVarErrorDf.loc[index]['R^2'], gVarErrorDf.loc[index]['MAE'], gVarErrorDf.loc[index]['RMSE %'], gVarErrorDf.loc[index]['R^2 %'], gVarErrorDf.loc[index]['MAE %']]
    #Data frame containing all single variable impacts
    
    impact_df.loc[8] = ["None dropped",'', gVarErrorDf.loc[0]['RMSE'], gVarErrorDf.loc[0]['R^2'], gVarErrorDf.loc[0]['MAE'], gVarErrorDf.loc[0]['RMSE %'], gVarErrorDf.loc[0]['R^2 %'], gVarErrorDf.loc[0]['MAE %']] 
    
    #double variable impact
    l = 9
    for j in range(7):
        #loop from j+1 to 8
        for k in range(j+1,8):
            if j != k:
                index = 2**j + 2**k #7- index to move through gVarNames correctly
                impact_df.loc[l] = [gVarNames[(7-j)], gVarNames[(7-k)], gVarErrorDf.loc[index]['RMSE'], gVarErrorDf.loc[index]['R^2'], gVarErrorDf.loc[index]['MAE'], gVarErrorDf.loc[index]['RMSE %'], gVarErrorDf.loc[index]['R^2 %'], gVarErrorDf.loc[index]['MAE %']]
                l += 1
    #Data frame containing all double variable impacts'''
    
    # Rank each metric, with the highest being the best for R^2 and the lowest being the best for RMSE and MAE
    impact_df['R^2_rank'] = impact_df['R^2'].rank(ascending=False)
    impact_df['RMSE_rank'] = impact_df['RMSE'].rank()
    impact_df['MAE_rank'] = impact_df['MAE'].rank()

    # Calculate the sum of the ranks
    impact_df['rank_sum'] = impact_df['R^2_rank'] + impact_df['RMSE_rank'] + impact_df['MAE_rank']

    # Find the model with the lowest rank sum
    best_model = impact_df['rank_sum'].idxmin()
    
    impact_df.sort_values(by=['rank_sum'], inplace=True) # Sort the data frame based on the rank sum
    impact_df.to_excel('impact_df.xlsx')#export to excel

## Final Predictions
    
Using all information gathered to create a retrained model (Found to be Random Forrest Regression) removing variables found to harm performance. Hyperparameters are re-optimised for this new data set. Results are displayed

In [ ]:
### Dropping variables found previously to have a negative impact on the model performance
def var_drop_final():
    """
    Drops columns from the original dataset based on a binary number.

    Each digit in the binary number corresponds to a column in the dataset. If the digit is 1, the corresponding column is dropped. If the digit is 0, the corresponding column is kept.

    Parameters:
    dropVar (int): A binary number where each digit represents a column in the dataset.

    Returns:
    pandas.DataFrame: The modified dataset with specified columns dropped. 
    """
    #sort gVarError by RMSE
    varError_sort = gVarErrorDf.sort_values(by=['RMSE'], ascending = True,  inplace=False)
    drop_string = varError_sort['Drop index'].iloc[0]

In [ ]:
### Testing the scaling and impuation of the data using the specific dropped variables
def scale_impute_final():
    #test each combination of imputer & scaler to find the best combination
    imputer = {
        "simple_imputer": SimpleImputer(strategy='mean'),
        'simple_imputer_median': SimpleImputer(strategy='median'),
        'simple_imputer_most_frequent': SimpleImputer(strategy='most_frequent'),
        'simple_imputer_constant': SimpleImputer(strategy='constant', fill_value=0),
        'iterative': IterativeImputer(max_iter=10, random_state=0),
        'mice': IterativeImputer(max_iter=10, random_state=0, estimator=BayesianRidge())
        }

    scaler = {
        'standard_scaler': StandardScaler(),
        'min_max_scaler': MinMaxScaler(),
        'max_abs_scaler': MaxAbsScaler(),
        'robust_scaler': RobustScaler(),
        'quantile_transformer_normal': QuantileTransformer(n_quantiles=700, output_distribution='normal'),
        'quantile_transformer_uniform': QuantileTransformer(n_quantiles=700, output_distribution='uniform'),
        'power_transformer': PowerTransformer(),
        'normalizer': Normalizer(),
        'binarizer': Binarizer()
        }

In [ ]:

def final_plot_predictions(final_model, X_test, y_test):
    """
    Plots y_pred vs y_test using the final model and displays a histogram of errors in y_pred.

    Parameters:
    final_model: The final trained model.
    X_test: The preprocessed testing dataset.
    y_test: The target variable for the testing dataset.

    Returns:
    None
    """

    # Plot y_pred vs y_test using the final model
    y_pred_final = final_model.predict(X_test)

    plt.scatter(y_test, y_pred_final)
    plt.xlabel('y_test')
    plt.ylabel('y_pred')
    plt.title('y_pred vs y_test using the final model')
    plt.show()

    errors = y_test - y_pred_final

    plt.hist(errors, bins=10)
    plt.xlabel('Errors')
    plt.ylabel('Frequency')
    plt.title('Histogram of Errors in y_pred')
    plt.show()

In [ ]:
def plot_learning_curve(estimator, X, y):
    """
    Plots the learning curve for an estimator.

    Parameters:
    estimator: The estimator model.
    X: The preprocessed training dataset.
    y: The target variable for the training dataset.

    Returns:
    None
    """
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=5, scoring='neg_mean_squared_error')

    train_scores_mean = -train_scores.mean(axis=1)
    test_scores_mean = -test_scores.mean(axis=1)

    plt.plot(train_sizes, train_scores_mean, label='Training error')
    plt.plot(train_sizes, test_scores_mean, label='Validation error')
    plt.xlabel('Training set size')
    plt.ylabel('Mean squared error')
    plt.title('Learning Curve')
    plt.legend()
    plt.show()


In [ ]:
def finalModel_histogram():
    """
    Plots a histogram of the errors in the final model's predictions.

    Parameters:
    None

    Returns:
    None
    """
    import matplotlib.pyplot as plt
    # Calculate the errors
    errors = gy_test - (random_forest_regression(gX_train_preprocessed, gy_train, gX_test_preprocessed))

    # Fit a normal distribution to the data
    mu, std = norm.fit(errors)

    # Create a histogram
    plt.hist(errors, bins=10, density=True)

    # Plot the PDF
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = norm.pdf(x, mu, std)
    plt.plot(x, p, 'k', linewidth=2)

    # Set labels and title
    plt.xlabel('Error')
    plt.ylabel('Frequency')
    plt.title('Histogram of Errors')

    #remove grid lines and add minor tick marks
    plt.grid(False)
    plt.minorticks_on()

    # Show the histogram
    plt.show()

In [ ]:
### Final error analysis

def final_error_analysis():
    # calculates uncertainties in the final model's predictions
    errors = gy_test - (random_forest_regression(gX_train_preprocessed, gy_train, gX_test_preprocessed))
    print("Mean error:", np.mean(errors))
    print("Standard deviation of errors:", np.std(errors))
    print("95% confidence interval of errors:", np.percentile(errors, [2.5, 97.5]))

    # calculates the percentage of predictions that are within 10% of the actual values
    errors = abs(errors)
    print("Percentage of predictions that are within 10% of the actual values:", sum(errors < 0.1 * gy_test) / len(errors))
    

## Graph Functions

1. `comp_graph()` - *Static*

    This function creates subplots allowing visual comparison between all regression functions.

2. `var_effect_line()` - *Interactive*

    This function is used to visualise the effect of a variable on the outcome. The user can choose any combination of variables to drop by deselecting a checkbox, the graph shows predicted vs actual values of Compressive strength, and the error metrics are shown rounded to 4 decimal places.

3. `var_effect_bar()` - *Interactive*

    This function is used to create a bar chart showing the effect on the various errors, of removing individual variables from the dataset. Users can choose the error type from the drop down menu.


In [ ]:
def comp_graph():
    """
    This function generates a comparative graph of actual vs predicted values for different regression models.

    The function calls the `regression_performance` function to get the best model and a DataFrame containing 
    the predictions of each model. It then creates a scatter plot for each model, comparing the actual and 
    predicted values. The scatter plots are arranged in a grid, with one subplot for each model with a y=x line added.

    The function doesn't return anything. The graph is displayed using `plt.show()`.

    Note: This function relies on the global variable `gy_test` for the actual values. Make sure this variable 
    is defined and properly initialized before calling this function.
    """
    # Number of models taking the data from the data frame in the regression performance function
    best_model, df = regression_performance()
    n_models = len(df)

    # Create a figure and axes with a subplot for each model
    fig, axs = plt.subplots(2, 4, figsize=(20, 15))

    # Flatten the axes array
    axs = axs.flatten()

    # Loop over each model
    for i, (model_name, row) in enumerate(df.iterrows()):         # Plot y_test vs y_pred
        axs[i].scatter(gy_test, row['y_pred'], s=10)
        axs[i].plot([gy_test.min(), gy_test.max()], [gy_test.min(), gy_test.max()], 'k--', lw=2)
        axs[i].set_xlabel('Actual CsMPa', fontsize=12)
        axs[i].set_ylabel('Predicted CsMPa', fontsize=12)
        axs[i].set_title(f'{model_name}')
        axs[i].xaxis.set_minor_locator(AutoMinorLocator())
        axs[i].yaxis.set_minor_locator(AutoMinorLocator())
        axs[i].grid(False)

    # Remove unused subplots
    for i in range(n_models, len(axs)):
        fig.delaxes(axs[i])

    # Add overall figure title
    fig.suptitle('Comparative Graph of Actual vs Predicted Values of Compressive Strength (CsMPa) \n (y=x line added for reference) \n', fontsize=15)
    
    plt.tight_layout()
    plt.show()

In [ ]:
def var_effect_line():
    """ 
    This function creates an interactive scatter plot of actual vs predicted values for the best model, with a trendline.
    Checkboxes are added to the plot, allowing the user to remove variables from the model. The plot is updated when the user clicks a checkbox.
    A text box is added to the plot, containing the effect of removing each variable on the error metrics.

    Parameters: None
    Returns: None (The plot is displayed using `plt.show()`)
    """

    # Create a list of checkboxes, taking the variable names from the global variable `gVarNames`
    checkboxes = [widgets.Checkbox(value=True, description=var) for var in gVarNames[0:8]]

    def update_lines(*args, **kwargs):
        """
        This function updates the scatter plot when the user clicks a checkbox.

        Parameters:
        *args: A list of arguments. The first argument is a boolean value indicating whether the checkbox is checked.
        **kwargs: A dictionary of keyword arguments. The key is the name of the checkbox, and the value is a boolean indicating whether the checkbox is checked.

        Returns: None
        """
        active_vars = [var.description for var in checkboxes if var.value] # Get the names of the active variables
        # Checks that at least one checkbox (variable) is selected, if not an error is printed and the function returns
        if not active_vars:
            print("Error: At least one checkbox must be selected.")
            return
        
        clear_output(wait=True) # Clear the output of the cell
        sns.set_style("whitegrid") # Set the style of the plot
        
        fig, ax = plt.subplots(figsize=(10, 6)) # Create a figure and axes
        plt.size=(10, 6)

        fig.subplots_adjust(left=0.1, right=0.9, bottom=0.1, top=0.9)  # Center the plot
        active_vars = [var.description for var in checkboxes if var.value]
        binary_string = ''.join(['1' if not checkbox.value else '0' for checkbox in checkboxes]) # Create a binary string representing the active variables
        print(f'Drop Index: {binary_string}')
        binary_int = int(binary_string,2)

        x = gy_test
        y = gVarErrorDf.loc[(binary_int)].iloc[1] # Using the binary string to select the correct row in the global DataFrame `gVarErrorDf`
        ax.scatter(x, y, label='Data')  # Add a label to the scatter plot
        
        # Fit a line to the data
        z = np.polyfit(x, y, 1)
        p = np.poly1d(z)
        
        # Add the line to the plot
        ax.plot(x, p(x), "r--", label='Trendline')  # Add a label to the trendline
        
        plt.xticks(fontsize=14)  # Increase the size of the x-axis labels
        plt.yticks(fontsize=14)  # Increase the size of the y-axis labels
        
        # Add titles & axis titles
        plt.title('Scatter plot of test against predicted values of \n compressive strength (CsMPa) ', fontsize=16) 
        plt.xlabel('Actual CsMPa', fontsize=14)
        plt.ylabel('Predicted CsMPa', fontsize=14)

        # Add legend
        plt.legend(bbox_to_anchor=(0.95, 0.2))
      
        ax = plt.gca() # Set ax = the current axes
        plt.minorticks_on()
        ax.grid(False)
        
        
        # Adding a text box containing each error metric rounded to 4 decimal places
        rmse = round(gVarErrorDf.loc[binary_int].iloc[2], 4)
        r2 = round(gVarErrorDf.loc[binary_int].iloc[3], 4)
        mae = round(gVarErrorDf.loc[binary_int].iloc[4], 4)
        
        text_box = f'RMSE = {rmse}\nR^2 = {r2}\nMAE = {mae}'
        plt.text(0.05, 0.95, text_box, transform=ax.transAxes, verticalalignment='top', fontsize=12, bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.5'))
        
        textstr ='''This scatter plot shows the relationship between the actual and 
        predicted values of Compressive Strength. A trendline is shown in red. 
        Each point on the plot represents a data point. The position of
        a point on the x-axis represents its actual value, and the position on 
        the y-axis represents its predicted value. The text box in the top left 
        corner shows the effect of removing each variable on the error metrics. 
        The RMSE, R^2, and MAE values are calculated with each variable removed, 
        and are rounded to 4 decimal places. RMSE is the Root Mean Square Error,
        a measure of the differences between values predicted by a model and the 
        values actually observed.'''
        
        # Add a text box containing the description of the plot
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        ax.text(1.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
        
        plt.show()
        

    # Grid of checkboxes in a 4x2 grid
    grid = [
        [checkboxes[i] for i in range(j, j+4)]
        for j in range(0, 8, 4)
    ]

    grid = [[checkboxes[i] for i in range(j, j + 4)] for j in range(0, 8, 4)]
    ui = widgets.VBox([widgets.HBox(row) for row in grid])
    out = widgets.interactive_output(update_lines, {checkbox.description: checkbox for checkbox in checkboxes})

    display(ui, out)

In [ ]:
def var_effect_bar():
    """
    This function generates an interactive bar chart, showing the effect of removing each variable on the performance metrics (RMSE, R^2, and MAE).
    Data is taken from the global DataFrame `gVarErrorDf`, which is populated by the `var_performance` function.

    A dropdown menu is used to select the performance metric to display. The bar chart shows the performance metric for each variable, as well as the baseline (i.e., when no variables are removed).

    Parameters:    None
    Returns:    None (displays a bar chart)
    """

    # List of binary variable names
    index = [bin(2**i) for i in range(8)] #index = ['0b1', '0b10'...
    
    # List of error types
    error_types = ['RMSE', 'R^2', 'MAE']

    def plot_bar_chart(error_type):
        # Get the values for each variable from gVarErrorDf
        plt.figure(figsize=(10, 6))
        global gVarNames
        values = []
        for name in index:
            row = gVarErrorDf.loc[gVarErrorDf['Drop index'] == name, error_type]
            if len(row) > 0:
                values.append(row.values[0])
            else:
                values.append(np.nan)  # replace None with np.nan
        

        plt.bar(gVarNames[0:8], values[::-1]) #Corrects order of names vs values, do to drop index being in reverse order
        plt.title(f'Bar chart of {error_type} after removing variable')
        plt.xlabel('Variable', fontsize=14, labelpad=10)
        plt.ylabel(error_type, fontsize=14)

        # Add a constant line for the baseline (no variables removed)
        baseline = gVarErrorDf.loc[gVarErrorDf['Drop index'] == '0b0', error_type].values[0]
        plt.axhline(y=baseline, color='r', linestyle='--')

        plt.legend(['Baseline', 'Variable'])
        plt.xticks(rotation=45)
        
        plt.grid(False)
        ax = plt.gca()
        ax.minorticks_on()
        
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        
    
        textstr = 'This graph shows the effect of removing each variable on the ' + error_type + '.\n' \
              'The red dashed line represents the baseline (no variables removed).\n' \
              'Use the dropdown menu to select the error type.'
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
        plt.text(1.05, 0.95, textstr, transform=ax.transAxes, fontsize=14,
            verticalalignment='top', bbox=props)
        plt.show()

    # Create dropdown menu for error types
    interact(plot_bar_chart, error_type=error_types)

In [ ]:
def visualize_rf_trees(rf_model, num_trees=1):
    """
    This function visualizes the first 'num_trees' trees in a Random Forest model.

    Parameters:
    rf_model (sklearn.ensemble.RandomForestRegressor): The trained Random Forest model.
    num_trees (int): The number of trees to visualize. Default is 1.
    """
    for i in range(num_trees):
        plt.figure(figsize=(20,10))  # Set the figure size
        plot_tree(rf_model.estimators_[i], filled=True)  # Plot the i-th decision tree
        plt.show()

#rf_model, y_pred = random_forest_regression(gX_train_preprocessed, gy_train, gX_test_preprocessed)
#visualize_rf_trees(rf_model, num_trees=3)

## Function: main()

The `main()` function is the entry point of the program. It is responsible for coordinating the execution of other functions and controlling the flow of the program.

In [ ]:
def main():
    X,y = csv_import()
    preprocessing(X,y)
    print("Preprocessing done")
    comp_graph()
    best_model, df = regression_performance()
    print("The best model is: ", best_model)
    #hyperparameter 
    print("Performing optimisation via grid search")
    best_grid_params = grid_search()
    print("Grid search complete: Best parameters:", best_grid_params, "\nErrors are as follows:")
    best_grid_params = opt_unopt_random_forest(best_grid_params)
    #var_performance
    var_performance(best_grid_params)
    gVarErrorDf.to_csv('var_performance.csv')
    print (gVarErrorDf)
    var_impact_pct()
    #var_drop_final()
    #print("Var performance done") 
    var_effect_line() 
    var_effect_bar()
main()